In [ ]:
# Import data
import pandas as pd
file_path = "../data/hasil_preprocessing.xlsx"
df = pd.read_excel(file_path)
df

In [ ]:
df_date = df.loc[:,[ "date"]]
df_date

,date
0,Tidak ada tanggal
1,25 Maret 2025
2,30 April 2025
3,Tidak ada tanggal
4,1 Januari 2025
...,...
149,1 Januari 2025
150,22 Maret 2025
151,17 Januari 2025
152,24 April 2025


In [ ]:
# Fungsi untuk mengubah format tanggal jika formatnya sesuai
def convert_if_datetime(value):
    try:
        # Cek apakah value bisa dikonversi ke datetime
        date_obj = pd.to_datetime(value, errors='raise')
        return date_obj.strftime('%d-%m-%Y')
    except Exception:
        # Jika gagal, kembalikan nilai aslinya
        return value

# Terapkan fungsi ke seluruh DataFrame
df_date['menangani_tanggalISO'] = df_date['date'].apply(convert_if_datetime)

# Menampilkan DataFrame
print(df_date)

                  date menangani_tanggalISO
0    Tidak ada tanggal    Tidak ada tanggal
1        25 Maret 2025        25 Maret 2025
2        30 April 2025           30-04-2025
3    Tidak ada tanggal    Tidak ada tanggal
4       1 Januari 2025       1 Januari 2025
..                 ...                  ...
149     1 Januari 2025       1 Januari 2025
150      22 Maret 2025        22 Maret 2025
151    17 Januari 2025      17 Januari 2025
152      24 April 2025           24-04-2025
153      25 Maret 2025        25 Maret 2025

[154 rows x 2 columns]


In [ ]:
import pandas as pd
from datetime import datetime
import re

# Dictionary untuk mengganti bulan Indonesia ke Bahasa Inggris
bulan_indo_english = {
    'Januari': 'January',
    'Februari': 'February',
    'Maret': 'March',
    'April': 'April',
    'Mei': 'May',
    'Juni': 'June',
    'Juli': 'July',
    'Agustus': 'August',
    'September': 'September',
    'Oktober': 'October',
    'November': 'November',
    'Desember': 'December'
}

# Fungsi untuk mengubah nama bulan Indonesia ke bulan Inggris dan format dd-mm-yyyy
def convert_indonesia_to_english(date_str):
    # Ganti nama bulan Indonesia dengan bulan dalam bahasa Inggris
    for bulan_indo, bulan_eng in bulan_indo_english.items():
        if bulan_indo in date_str:
            date_str = date_str.replace(bulan_indo, bulan_eng)
            break  # Hanya perlu mengganti satu bulan, berhenti setelahnya
    
    # Mengonversi ke objek datetime
    date_obj = datetime.strptime(date_str, '%d %B %Y')
    
    # Mengubah ke format dd-mm-yyyy
    return date_obj.strftime('%d-%m-%Y')

# Fungsi untuk memeriksa apakah format tanggal sesuai
def is_valid_date(date_str):
    # Regex untuk mencocokkan format "dd bulan yyyy" dengan bulan dalam Bahasa Indonesia
    return bool(re.match(r'^\d{1,2} (Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember) \d{4}$', date_str))

# Terapkan fungsi ke kolom 'menangani_tanggalISO' hanya jika format valid
df_date['formatted_date(DD-MM-YYYY)'] = df_date['menangani_tanggalISO'].apply(
    lambda x: convert_indonesia_to_english(x) if is_valid_date(x) else x  # Kembalikan nilai awal jika tidak valid
)

# Menampilkan DataFrame
print(df_date)


                  date menangani_tanggalISO formatted_date(DD-MM-YYYY)
0    Tidak ada tanggal    Tidak ada tanggal          Tidak ada tanggal
1        25 Maret 2025        25 Maret 2025                 25-03-2025
2        30 April 2025           30-04-2025                 30-04-2025
3    Tidak ada tanggal    Tidak ada tanggal          Tidak ada tanggal
4       1 Januari 2025       1 Januari 2025                 01-01-2025
..                 ...                  ...                        ...
149     1 Januari 2025       1 Januari 2025                 01-01-2025
150      22 Maret 2025        22 Maret 2025                 22-03-2025
151    17 Januari 2025      17 Januari 2025                 17-01-2025
152      24 April 2025           24-04-2025                 24-04-2025
153      25 Maret 2025        25 Maret 2025                 25-03-2025

[154 rows x 3 columns]


In [ ]:
import re
from dateutil import parser

def extract_and_standardize_date(text):
    # 1. Bersihkan nama hari dan timezone/waktu
    text_cleaned = re.sub(r'\b(Senin|Selasa|Rabu|Kamis|Jumat|Sabtu|Minggu)\b,?\s*', '', text, flags=re.IGNORECASE)
    text_cleaned = re.sub(r'\|\s*', ' ', text_cleaned)  # ubah | menjadi spasi
    text_cleaned = re.sub(r'\b(WIB|am|pm|AM|PM)\b', '', text_cleaned, flags=re.IGNORECASE)
    text_cleaned = text_cleaned.strip()

    # 2. Semua kemungkinan regex tanggal
    date_patterns = [
        r'\d{1,2} [A-Za-z]+ \d{4}',                              # 26 February 2025
        r'\d{1,2} [A-Za-z]+ \d{4}, \d{2}:\d{2}',                  # 3 Feb 2025, 12:44
        r'\d{1,2} [A-Za-z]+ \d{4} \d{2}:\d{2}',                   # 3 Feb 2025 12:44
        r'\d{2}/\d{2}/\d{4}',                                    # 03/04/2025
        r'\d{4}-\d{2}-\d{2}',                                    # 2025-04-03
    ]

    # 3. Coba cari pola tanggal
    for pattern in date_patterns:
        match = re.search(pattern, text_cleaned)
        if match:
            try:
                dt = parser.parse(match.group(0), dayfirst=True, fuzzy=True)
                return dt.strftime('%d-%m-%Y')
            except Exception:
                continue

    # 4. Jika tidak cocok dengan pola, coba parsing langsung dari hasil pembersihan
    try:
        dt = parser.parse(text_cleaned.strip(), dayfirst=True, fuzzy=True)
        return dt.strftime('%d-%m-%Y')
    except Exception:
        return text  # jika gagal parsing, kembalikan teks asli

df_date['menangani_kalimat'] = df_date['formatted_date(DD-MM-YYYY)'].apply(extract_and_standardize_date)

In [ ]:
import pandas as pd
import re

# Fungsi untuk memeriksa format tanggal dd-mm-yyyy
def check_date_format(date_str):
    # Regex untuk format dd-mm-yyyy
    pattern = r'^\d{2}-\d{2}-\d{4}$'
    if re.match(pattern, date_str):
        return date_str
    else:
        return 'Tidak ada tanggal'

# Terapkan fungsi ke kolom 'menangani_kalimat'
df_date['date_final'] = df_date['menangani_kalimat'].apply(check_date_format)

print(df_date)


                  date menangani_tanggalISO formatted_date(DD-MM-YYYY)  \
0    Tidak ada tanggal    Tidak ada tanggal          Tidak ada tanggal   
1        25 Maret 2025        25 Maret 2025                 25-03-2025   
2        30 April 2025           30-04-2025                 30-04-2025   
3    Tidak ada tanggal    Tidak ada tanggal          Tidak ada tanggal   
4       1 Januari 2025       1 Januari 2025                 01-01-2025   
..                 ...                  ...                        ...   
149     1 Januari 2025       1 Januari 2025                 01-01-2025   
150      22 Maret 2025        22 Maret 2025                 22-03-2025   
151    17 Januari 2025      17 Januari 2025                 17-01-2025   
152      24 April 2025           24-04-2025                 24-04-2025   
153      25 Maret 2025        25 Maret 2025                 25-03-2025   

     menangani_kalimat         date_final  
0    Tidak ada tanggal  Tidak ada tanggal  
1           25-03-2025 

In [ ]:
# Simpan DataFrame
output_path = "../data/hasildate.xlsx"
df_date.to_excel(output_path, index=False)

In [ ]:
# Misalkan df_date sudah memiliki kolom 'date_final' dan df adalah DataFrame yang sudah ada
df['date_final'] = df_date['date_final']

# Menampilkan df setelah kolom 'date_final' dimasukkan
df

,NO,SUMBER,PENERBIT,LINK,JUDUL,content,tags,date,status,clean_text,content_clean,tokens,stopword_removal,stemming_output,rareword_removed,date_final,konten_final
0,2,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,dirjen pajak ungkap kabar terbaru soal perbaik...,direktur jenderal (dirjen) pajak suryo utomo m...,"coretax,dirjen pajak,administrasi perpajakan,f...",Tidak ada tanggal,success,direktur jenderal pajak suryo utomo mengungka...,direktur jenderal pajak suryo utomo mengungkap...,"['direktur', 'jenderal', 'pajak', 'suryo', 'ut...","['direktur', 'jenderal', 'pajak', 'suryo', 'ut...","['direktur', 'jenderal', 'pajak', 'suryo', 'ut...","['direktur', 'jenderal', 'pajak', 'suryo', 'ut...",Tidak ada tanggal,"[ ' d i r e k t u r ' , ' j e n d e r a l ' ..."
1,3,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,djp klaim aplikasi coretax mulai stabil digunakan,direktorat jenderal pajak (djp) kementerian ke...,"coretax,djp,aplikasi perpajakan,pembaruan,ppnb...",25 Maret 2025,success,"rata berada di bawah , detik, dengan performa ...",rata berada di bawah detik dengan performa ter...,"['rata', 'berada', 'di', 'bawah', 'detik', 'de...","['rata', 'berada', 'bawah', 'detik', 'performa...","['rata', 'ada', 'bawah', 'detik', 'performa', ...","['rata', 'ada', 'bawah', 'detik', 'performa', ...",25-03-2025,"[ ' r a t a ' , ' a d a ' , ' b a w a h ' ..."
2,5,local news,DDTC,https://news.ddtc.co.id/berita/nasional/181042...,"latensi turun, djp klaim coretax system sudah ...","ilustrasi. jakarta, ddtcnews - ditjen pajak (d...","administrasi pajak, coretax, coretax system, djp",30 April 2025,success,ditjen pajak menilai performa coretax adminis...,ditjen pajak menilai performa coretax administ...,"['ditjen', 'pajak', 'menilai', 'performa', 'co...","['ditjen', 'pajak', 'menilai', 'performa', 'co...","['ditjen', 'pajak', 'nilai', 'performa', 'core...","['ditjen', 'pajak', 'nilai', 'performa', 'core...",30-04-2025,"[ ' d i t j e n ' , ' p a j a k ' , ' n i ..."
3,6,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,awas penipuan catut nama coretax! begini modus...,ditjen pajak (djp) kementerian keuangan (kemen...,"penipuan,coretax,ditjen pajak,catut nama coret...",Tidak ada tanggal,success,akhir ini marak penipuan yang mengatasnamakan ...,akhir ini marak penipuan yang mengatasnamakan ...,"['akhir', 'ini', 'marak', 'penipuan', 'yang', ...","['akhir', 'marak', 'penipuan', 'mengatasnamaka...","['akhir', 'marak', 'tipu', 'mengatasnamakan', ...","['akhir', 'marak', 'tipu', 'mengatasnamakan', ...",Tidak ada tanggal,"[ ' a k h i r ' , ' m a r a k ' , ' t i p ..."
4,7,local news,Tempo,https://www.tempo.co/ekonomi/hari-ini-direktor...,hari ini direktorat jenderal pajak rapat tertu...,scroll ke bawah untuk membaca berita baca be...,tidak ada tags,1 Januari 2025,success,direktur jenderal atau dirjen pajak kementeria...,direktur jenderal atau dirjen pajak kementeria...,"['direktur', 'jenderal', 'atau', 'dirjen', 'pa...","['direktur', 'jenderal', 'dirjen', 'pajak', 'k...","['direktur', 'jenderal', 'dirjen', 'pajak', 'm...","['direktur', 'jenderal', 'dirjen', 'pajak', 'm...",01-01-2025,"[ ' d i r e k t u r ' , ' j e n d e r a l ' ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,155,local news,CNBC,https://www.cnbcindonesia.com/news/20250114071...,keluhan soal coretax terus muncul: sistem tak ...,"jakarta, cnbc indonesia - kalangan pengusaha d...","coretax, pajak, djp",1 Januari 2025,success,kalangan pengusaha dan pakar pajak memberikan ...,kalangan pengusaha dan pakar pajak memberikan ...,"['kalangan', 'pengusaha', 'dan', 'pakar', 'paj...","['kalangan', 'pengusaha', 'pakar', 'pajak', 'm...","['kalang', 'usaha', 'pakar', 'pajak', 'beri', ...","['kalang', 'usaha', 'pakar', 'pajak', 'beri', ...",01-01-2025,"[ ' k a l a n g ' , ' u s a h a ' , ' p a ..."
150,156,local news,Viva,https://www.viva.co.id/bisnis/1809316-coretax-...,"coretax bermasalah di awal tahun, misbakhun te...","jakarta

In [ ]:
print(df["rareword_removed"].iloc[0])
print(type(df["rareword_removed"].iloc[0]))

['direktur', 'jenderal', 'pajak', 'suryo', 'utomo', 'ungkap', 'kabar', 'baru', 'baik', 'sistem', 'aplikasi', 'administrasi', 'paja', 'coretax', 'adapun', 'coretax', 'sendiri', 'sempat', 'alami', 'kendala', 'sejak', 'implementasi', 'mulai', 'bulan', 'januari', 'suryo', 'aku', 'memang', 'temu', 'jumlah', 'kendala', 'operasi', 'coretax', 'tanah', 'sejak', 'januari', 'lalu', 'beberapa', 'antara', 'mulai', 'sulit', 'masalah', 'terbit', 'surat', 'pemberitahuan', 'tahun', 'hingga', 'terbit', 'faktur', 'pajak', 'sejak', 'implementasi', 'januari', 'kemarin', 'memang', 'betul', 'temu', 'beberapa', 'kendala', 'terap', 'terus', 'laku', 'baik', 'kena', 'buat', 'waktu', 'waktu', 'alami', 'baik', 'januari', 'sekarang', 'kata', 'suryo', 'konferensi', 'pers', 'apbn', 'kantor', 'jakarta', 'rabu', 'scroll', 'continue', 'with', 'content', 'suryo', 'jelas', 'jumlah', 'masalah', 'jadi', 'prioritas', 'utama', 'selesai', 'segera', 'sistem', 'coretax', 'salah', 'satu', 'ialah', 'latensi', 'waktu', 'tunggu', 'a

In [ ]:
import ast

# Convert string ke list (jika perlu)
df["rareword_removed"] = df["rareword_removed"].apply(ast.literal_eval)

In [ ]:
df["konten_final"] = df["rareword_removed"].apply(lambda tokens: " ".join(tokens))
df

,NO,SUMBER,PENERBIT,LINK,JUDUL,content,tags,date,status,clean_text,content_clean,tokens,stopword_removal,stemming_output,rareword_removed,date_final,konten_final
0,2,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,dirjen pajak ungkap kabar terbaru soal perbaik...,direktur jenderal (dirjen) pajak suryo utomo m...,"coretax,dirjen pajak,administrasi perpajakan,f...",Tidak ada tanggal,success,direktur jenderal pajak suryo utomo mengungka...,direktur jenderal pajak suryo utomo mengungkap...,"['direktur', 'jenderal', 'pajak', 'suryo', 'ut...","['direktur', 'jenderal', 'pajak', 'suryo', 'ut...","['direktur', 'jenderal', 'pajak', 'suryo', 'ut...","[direktur, jenderal, pajak, suryo, utomo, ungk...",Tidak ada tanggal,direktur jenderal pajak suryo utomo ungkap kab...
1,3,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,djp klaim aplikasi coretax mulai stabil digunakan,direktorat jenderal pajak (djp) kementerian ke...,"coretax,djp,aplikasi perpajakan,pembaruan,ppnb...",25 Maret 2025,success,"rata berada di bawah , detik, dengan performa ...",rata berada di bawah detik dengan performa ter...,"['rata', 'berada', 'di', 'bawah', 'detik', 'de...","['rata', 'berada', 'bawah', 'detik', 'performa...","['rata', 'ada', 'bawah', 'detik', 'performa', ...","[rata, ada, bawah, detik, performa, baik, cata...",25-03-2025,rata ada bawah detik performa baik catat detik...
2,5,local news,DDTC,https://news.ddtc.co.id/berita/nasional/181042...,"latensi turun, djp klaim coretax system sudah ...","ilustrasi. jakarta, ddtcnews - ditjen pajak (d...","administrasi pajak, coretax, coretax system, djp",30 April 2025,success,ditjen pajak menilai performa coretax adminis...,ditjen pajak menilai performa coretax administ...,"['ditjen', 'pajak', 'menilai', 'performa', 'co...","['ditjen', 'pajak', 'menilai', 'performa', 'co...","['ditjen', 'pajak', 'nilai', 'performa', 'core...","[ditjen, pajak, nilai, performa, coretax, admi...",30-04-2025,ditjen pajak nilai performa coretax administra...
3,6,economic,Detik,https://finance.detik.com/berita-ekonomi-bisni...,awas penipuan catut nama coretax! begini modus...,ditjen pajak (djp) kementerian keuangan (kemen...,"penipuan,coretax,ditjen pajak,catut nama coret...",Tidak ada tanggal,success,akhir ini marak penipuan yang mengatasnamakan ...,akhir ini marak penipuan yang mengatasnamakan ...,"['akhir', 'ini', 'marak', 'penipuan', 'yang', ...","['akhir', 'marak', 'penipuan', 'mengatasnamaka...","['akhir', 'marak', 'tipu', 'mengatasnamakan', ...","[akhir, marak, tipu, mengatasnamakan, implemen...",Tidak ada tanggal,akhir marak tipu mengatasnamakan implementasi ...
4,7,local news,Tempo,https://www.tempo.co/ekonomi/hari-ini-direktor...,hari ini direktorat jenderal pajak rapat tertu...,scroll ke bawah untuk membaca berita baca be...,tidak ada tags,1 Januari 2025,success,direktur jenderal atau dirjen pajak kementeria...,direktur jenderal atau dirjen pajak kementeria...,"['direktur', 'jenderal', 'atau', 'dirjen', 'pa...","['direktur', 'jenderal', 'dirjen', 'pajak', 'k...","['direktur', 'jenderal', 'dirjen', 'pajak', 'm...","[direktur, jenderal, dirjen, pajak, menteri, u...",01-01-2025,direktur jenderal dirjen pajak menteri uang su...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,155,local news,CNBC,https://www.cnbcindonesia.com/news/20250114071...,keluhan soal coretax terus muncul: sistem tak ...,"jakarta, cnbc indonesia - kalangan pengusaha d...","coretax, pajak, djp",1 Januari 2025,success,kalangan pengusaha dan pakar pajak memberikan ...,kalangan pengusaha dan pakar pajak memberikan ...,"['kalangan', 'pengusaha', 'dan', 'pakar', 'paj...","['kalangan', 'pengusaha', 'pakar', 'pajak', 'm...","['kalang', 'usaha', 'pakar', 'pajak', 'beri', ...","[kalang, usaha, pakar, pajak, beri, catat, khu...",01-01-2025,kalang usaha pakar pajak beri catat khusus per...
150,156,local news,Viva,https://www.viva.co.id/bisnis/1809316-coretax-...,"coretax bermasalah di awal tahun, misbakhun te...",

In [ ]:
# Simpan DataFrame
output_path = "../data/hasil_preprocessing2.xlsx"
df.to_excel(output_path, index=False)